In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('transformed_v2.csv', index_col=0)

In [3]:
df.head()

,duration,sex,age,country,education,undergrad_major,role,industry,experience,salary,...,Q50_Part_5,Q50_Part_6,Q50_Part_7,Q50_Part_8,"Q12_MULTIPLE_CHOICE_Advanced statistical software (SPSS, SAS, etc.)","Q12_MULTIPLE_CHOICE_Basic statistical software (Microsoft Excel, Google Sheets, etc.)","Q12_MULTIPLE_CHOICE_Business intelligence software (Salesforce, Tableau, Spotfire, etc.)","Q12_MULTIPLE_CHOICE_Cloud-based data software & APIs (AWS, GCP, Azure, etc.)","Q12_MULTIPLE_CHOICE_Local or hosted development environments (RStudio, JupyterLab, etc.)",Q12_MULTIPLE_CHOICE_Other
2,434,Male,30-34,Indonesia,Bachelor’s degree,Engineering (non-computer focused),Other,Manufacturing/Fabrication,5-10,"10-20,000",...,0,0,0,0,0,1,0,0,0,0
3,718,Female,30-34,United States of America,Master’s degree,"Computer science (software engineering, etc.)",Data Scientist,I am a student,0-1,"0-10,000",...,0,0,0,0,0,0,0,0,1,0
5,731,Male,22-24,India,Master’s degree,Mathematics or statistics,Data Analyst,I am a student,0-1,"0-10,000",...,1,0,0,0,1,0,0,0,0,0
7,959,Male,35-39,Chile,Doctoral degree,"Information technology, networking, or system ...",Other,Academics/Education,10-15,"10-20,000",...,0,1,0,0,0,1,0,0,0,0
8,1758,Male,18-21,India,Master’s degree,"Information technology, networking, or system ...",Other,Other,0-1,"0-10,000",...,1,0,0,0,0,0,0,0,1,0


In [4]:
df_tf = pd.get_dummies(df[[c for c in df.columns if c != 'salary']], dummy_na=True)

In [5]:
target = df['salary']

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [7]:
lr = LogisticRegression(random_state=42, solver='lbfgs', multi_class='auto')

In [12]:
from sklearn.model_selection import cross_val_score
cross_val_score(lr, df_tf, target, cv=5).mean()

/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs faile

0.28187836903996133

In [8]:
rf = RandomForestClassifier(random_state=42, n_estimators=100)

In [9]:
df_tf.shape

(12956, 660)

In [19]:
cross_val_score(rf, df_tf, target, cv=5).mean()

KeyboardInterrupt: 

In [22]:
rf.fit(df_tf, target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [20]:
import numpy as np

In [23]:
imp_sorted = sorted(rf.feature_importances_, reverse=True)
feature_names = df_tf.columns[np.argsort(rf.feature_importances_)[::-1]]

In [32]:
aggregates = {}
counts = {}

In [33]:
for feat, imp in zip(feature_names, np.sqrt(imp_sorted)):
    feat_part = feat.split("_")[0]
    if feat_part in aggregates:
        aggregates[feat_part] += imp
        counts[feat_part] += 1
    else:
        aggregates[feat_part] = imp
        counts[feat_part] = 1

In [49]:
# Aggregation by summing up
sorted_x = sorted(aggregates.items(), key=lambda kv: kv[1], reverse=True)
sorted_x

[('country', 1.369362099730011),
 ('Q28', 0.8613016745731088),
 ('Q38', 0.8063115747419172),
 ('Q19', 0.801833689981759),
 ('Q13', 0.764944396057886),
 ('role', 0.7417983866798459),
 ('industry', 0.7413897946908695),
 ('Q29', 0.7376866627851327),
 ('Q16', 0.7306828206781087),
 ('Q47', 0.643267761807991),
 ('Q27', 0.5878641884773331),
 ('age', 0.5795149695816382),
 ('Q31', 0.5677256861558477),
 ('Q33', 0.5620865530928003),
 ('Q30', 0.5611860740755878),
 ('Q49', 0.5580391228845492),
 ('Q36', 0.5510922306335196),
 ('Q21', 0.5484704343433225),
 ('experience', 0.501033040211988),
 ('undergrad', 0.4952744709607372),
 ('Q17', 0.451273712937471),
 ('Q20', 0.4449118034836645),
 ('Q46', 0.4281335034168396),
 ('Q32', 0.4134818243249075),
 ('Q24', 0.40603186390497886),
 ('Q11', 0.4038542825102086),
 ('Q14', 0.40334200422019384),
 ('Q25', 0.3935997579651797),
 ('Q43', 0.37551722459483955),
 ('Q37', 0.37128314080168934),
 ('Q10', 0.36150359401704873),
 ('Q50', 0.33796591835084516),
 ('Q22', 0.323193

In [48]:
# Aggregation by mean
means = {v: sum / count for v, sum, count in zip(aggregates.keys(), aggregates.values(), counts.values())}
sorted_x = sorted(means.items(), key=lambda kv: kv[1], reverse=True)
sorted_x

[('duration', 0.13602572552800477),
 ('Q11', 0.0576934689300298),
 ('Q10', 0.0516433705738641),
 ('Q33', 0.05109877755389094),
 ('Q13', 0.050996293070525736),
 ('Q45', 0.048393949828701786),
 ('Q31', 0.047310473846320644),
 ('Q26', 0.0470335007840079),
 ('Q12', 0.046729213865887916),
 ('Q44', 0.04656110885508557),
 ('Q49', 0.046503260240379096),
 ('Q15', 0.04554494396730059),
 ('age', 0.04457807458320294),
 ('Q42', 0.04401271535689093),
 ('Q23', 0.04254909535517908),
 ('Q36', 0.04239171004873228),
 ('Q50', 0.042245739793855645),
 ('Q19', 0.042201773156934685),
 ('Q21', 0.04219003341102481),
 ('Q40', 0.04193513159857877),
 ('Q48', 0.04188889242093425),
 ('experience', 0.041752753350999004),
 ('Q16', 0.04059349003767271),
 ('Q47', 0.040204235112999434),
 ('industry', 0.03706948973454348),
 ('Q24', 0.03691198762772535),
 ('Q14', 0.03666745492910853),
 ('Q38', 0.0366505261246326),
 ('Q25', 0.0357817961786527),
 ('Q46', 0.0356777919514033),
 ('undergrad', 0.03537674792576694),
 ('role', 0.0